In [2]:
#抽象类加抽象方法就等于面向对象编程中的接口
from abc import ABCMeta,abstractmethod
from math import exp, sqrt
from random import gauss
from WindPy import w
import datetime
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [3]:
class StockPriceEngine(object):
    __metaclass__ = ABCMeta #指定这是一个抽象类
    @abstractmethod  #抽象方法
    def getPrice(self):
        pass

In [4]:
class WindData(StockPriceEngine):
    def __init__(self, startDate, endDate, stockCode):    
        self.startDate = startDate
        self.endDate = endDate
        self.stockCode = stockCode    
    
    def getPrice(self):
        """
        获取某个指数某一段时间的收盘价格，返回值为WindData

        参数：startDate：起始日期，字符串格式
               endDate：终止日期，字符串格式
             benchmark：需要查询的指数或者股票代码，字符串格式

        返回值：WindData，e.g.
              .ErrorCode=0
              .Codes=[000905.SH]
              .Fields=[CLOSE]
              .Times=[20201201,20201202,20201203,20201204,20201207,20201208,20201209,20201210,20201211]
              .Data=[[6426.6155,6434.9791,6440.8808,6464.7345,6424.6823,6423.6513,6307.2207,6320.0956,6209.427]]
        """
        w.isconnected()
        closingData = w.wsd(self.stockCode, "close", self.startDate, self.endDate, "").Data[0]
        return closingData

In [6]:
class MonteCarlo(StockPriceEngine):
    results = []
    
    def __init__(self, stockCode, firstClosingPrice, riskFreeRate, tradingDaysPerYear, stockVolatility, numberOfSimulation, dateList):    
        self.stockCode = stockCode
        self.firstClosingPrice = firstClosingPrice
        self.riskFreeRate = riskFreeRate
        self.tradingDaysPerYear = tradingDaysPerYear
        self.stockVolatility = stockVolatility
        self.numberOfSimulation = numberOfSimulation
        self.dateList = dateList
    
    def getPrice(self):
        for i in range(0, self.numberOfSimulation):
            result = self.getOnePredict(self.stockCode, self.firstClosingPrice, self.dateList, 
                                        self.riskFreeRate, self.tradingDaysPerYear, self.stockVolatility)
            self.results.append(result)
        return self.results     
    
    def getOnePredict(self, stockCode, firstClosingPrice, dateList, riskFreeRate, tradingDaysPerYear, stockVolatility):
        """
        对于某股票/指数，获取其在某一段时间每一天的预测价格
        stockCode：股票/指数代码
        firstClosingPrice：要预测的时间段，第一天的收盘价，取自wind
        dateList：要预测的时间段（包含上述的第一天）
        riskFreeRate：无风险利率
        tradingDaysPerYear：一年交易日天数
        """
        result = [firstClosingPrice]
        previousDayClosingPrice = firstClosingPrice

        for dateIndex in range(1, len(dateList)):
            testingDate_dt = dateList[dateIndex]
            predictedPrice = self.calculate_S_T(previousDayClosingPrice, self.stockVolatility, self.riskFreeRate, 1 / 365)
            result.append(predictedPrice)
            previousDayClosingPrice = predictedPrice
        return result
    
    def calculate_S_T(self, stockClosingPrice, stockPriceVolatility, riskFreeRate, daysToExpire):
        """模拟epsilon，计算S_T
        stockClosingPrice：测试当日的标的价格
        """
        return stockClosingPrice * exp((riskFreeRate - 0.5 * stockPriceVolatility ** 2) * daysToExpire + 
                                       stockPriceVolatility * sqrt(daysToExpire) * gauss(0.0, 1.0))

In [1]:
# """test"""
# STOCK_CODE = "000905.SH"

# """无风险利率 2020年国债一年期利率加权"""#http://www.chinamoney.com.cn/chinese/sddsintigy/
# RISK_FREE_RATE = 2.1927 / 100

# """蒙特卡洛模拟次数"""
# SIMULATIONS = 100000

# """一年交易日天数"""
# TRADING_DATES_PER_YEAR = 240

# """获取期权交易日序列"""

# OPTION_START_DATE = "2020-06-23"
# OPTION_STRIKE_DATE = "2021-03-05"
# dateList = w.tdays(OPTION_START_DATE, OPTION_STRIKE_DATE, "").Data[0]

# priceEngineObj1 = MonteCarlo(STOCK_CODE, 3000, RISK_FREE_RATE, TRADING_DATES_PER_YEAR, 0.02, 100, dateList)
# priceEngineObj1.getPrice()

# priceEngineObj2 = WindData(OPTION_START_DATE, OPTION_STRIKE_DATE, STOCK_CODE)
# priceEngineObj2.getPrice()